In [6]:
from gurobipy import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations
import matplotlib.pyplot as plt
import os, sys
 

In [12]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 2
        self.truckNum    = 2
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        self.model = None
      
        

    def readData(self, path):
        self.dt = pd.read_csv(path, header = None).to_numpy()[:-1]
        self.customerNum = len(self.dt) - 1
        self.i_pot = self.dt[0, 1:3]
        self.nodeNum = self.customerNum + 1
        self.cor_X = [self.dt[i, 1:3][0] for i in range(len(self.dt))]
        self.cor_Y = [self.dt[i, 1:3][1] for i in range(len(self.dt))]
        self.cities = [self.dt[i, 0] for i in range(len(self.dt))]
        
        self.cus_can_served_by_drone = [i for i in range(len(self.dt)) if self.dt[i, 3] == 0]
        
        self.drone_distances = [round(distance.euclidean((self.dt[i, 1:3]), self.i_pot),2)
                                if self.dt[i, 3] == 0 else float('inf')
                                for i in range(len(self.dt))]
        self.truck_distances = [[round(distance.cityblock(self.dt[i, 1:3], self.dt[j, 1:3]),1)
                                 for i in range(len(self.dt))] for j in range(len(self.dt))]
    
        for i in range(len(self.dt)):
            self.truck_distances[i].append(self.truck_distances[i][0])
        
        #Decision variables

        # x_ij if (i->j) in vehicle tour
        # # y_im = 1 if cus i assigned to drone m ()

        self.x = None  
        self.y = None 
    def addConstrs(self):


            #SET
            C = [i for i in range (1, self.customerNum + 1)]
            
            N = [i for i in range(self.customerNum + 2)]
            
            N0 = [i for i in range(self.customerNum + 1)]            
            
            N_plus = [i for i in range (1, self.customerNum + 2)]
            
            R = [i for i in range(self.truckNum)]
            U = [k for k in range(self.droneNum)]
            G = [0] + C
            C_U = data.cus_can_served_by_drone

            self.x = {}
            self.y = {}
            self.u = {}
            
            
            #1
            #completion time
            alpha = self.model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
            self.model.update()
            expr = LinExpr(0)
            expr.addTerms(1.0, alpha)

            self.model.setObjective(expr, GRB.MINIMIZE)
            expr.clear()
            #add vars u[i]
            for i in N_plus:
                self.u[i] = self.model.addVar( vtype = GRB.INTEGER, name = "u%d" %i)
            #2
            expr = LinExpr(0)
            for r in R:
                for i in N0:
                    for j in N_plus:
                        if i != j:

                            self.x[i,j,r] = self.model.addVar(0, 1, vtype = GRB.BINARY,name = "x%d,%d,%d" %(i,j,r))

                            self.model.update()
                                
                            expr.addTerms(self.truck_distances[i][j], self.x[i,j,r])
                        else:
                            self.x[i,i,r] = self.model.addVar(0.0, 1.0, 0.0, GRB.BINARY, "x%d,%d,%d" %(i,j,r))

        #         print(expr)

                self.model.addConstr(alpha >= expr, "truckTime")
                expr.clear()
            expr.clear()


            #3
            for k in U:
                expr = LinExpr(0)
                for i in C:

                    if i in C_U:
                        self.y[k,i] = self.model.addVar(0,1, vtype= GRB.BINARY, name =  "y%d,%d" %(k,i))
    #                     print(i)

                        expr.addTerms(self.drone_distances[i], self.y[k,i])
                    else:
                        self.y[k,i] = self.model.addVar(0, 0, vtype = GRB.BINARY, name = "y%d,%d" %(k,i))
                self.model.update()
    #             print(expr)
                self.model.addConstr(alpha >= expr, "dronetime")
                expr.clear()
            expr.clear()    
            
            
            #
            expr = LinExpr(0)
            for k in U:
                for j in C:
                    if j in C_U:
                        expr.addTerms(1.0, self.y[k,j])
                self.model.addConstr(expr <= 1)
                expr.clear()
            expr.clear()           
    
            #4
            for j in C:

                expr1 = LinExpr(0)
                expr2 = LinExpr(0)

                for i in N0:
                    if i != j:
                        for r in R:
                            expr1.addTerms(1.0, self.x[i,j,r])
    #                 print(expr1)

                if j in C_U:
                    for k in U:
                        expr2.addTerms(1.0, self.y[k,j])

    #             print(expr2)
    #             print("------------------")

                self.model.addConstr(expr1 + expr2 == 1, "served customer once")
                expr1.clear()
                expr2.clear()
            expr1.clear()
            expr2.clear()


            #5
#             for i in N0:

#                 expr1 = LinExpr(0)
#                 expr2 = LinExpr(0)

#                 for j in C:

#                     expr1.addTerms(1.0, self.x[i,j])
#     #             print(expr1)
#                 if i in C_U:
#                     for k in U:
#                         expr2.addTerms(1.0, self.y[k,i])

#     #             print(expr2)
#     #             print("------------------")
#                 self.model.addConstr(expr1 + expr2 == 1, "served customer once")
#                 expr1.clear()
#                 expr2.clear()
#             expr1.clear()
#             expr2.clear()


            #6
            for r in R:
                for j in C:
                
                    expr1 = LinExpr(0)
                    expr2 = LinExpr(0)

                    for i in N0:
                        if i != j:
                            expr1.addTerms(1.0, self.x[i,j,r])

                    for h in N_plus:
                        if h != j:
                            expr2.addTerms(1.0, self.x[j,h,r])
                    self.model.addConstr(expr1 == expr2, "flow conservation")
                    expr1.clear()
                    expr2.clear()
            expr1.clear()
            expr2.clear()

            #

#             #get all subtours
#             def powerset(iterable):
#                 "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
#                 s = list(iterable)
#                 return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

#             S = list(powerset(range(1, len(data.cities))))
#             # The first element of the list is the empty set and the last element is the full set, hence we remove them.
#             S = S[1:(len(S))]
#             S = [list(s) for s in S]

#     #         print(len(S))
#     #         import sys
#     #         print(sys.getsizeof(S)/1024/1024," GB")
#     #         print(type(S))



#             # for s in S:

#             #         s.insert(0,0)
#             S.insert(0,[0])
#             S = S[0:len(S) -1 ]
#             # print(S)
#             # S = [[0,1,2,3,4]]


#             #7
#             for s in S:
#                 expr1 = LinExpr(0)
#                 expr2 = LinExpr(0)

#                 for i in s:
#                     for j in V:
#                         if j not in s:
#                             expr1.addTerms(1.0, self.x[i,j])

#                     for k in U:
#                         if i in C_U:
#                             expr2.addTerms(1.0, self.y[k,i])
#                 self.model.update()
#     #     print(expr1)
#     #     print(expr2)
#                 self.model.addConstr(expr1 + expr2 >= 1)
#                 expr1.clear()
#                 expr2.clear()
                
            #8
            expr = LinExpr(0)
            for r in R:
                for j in N_plus:
                    expr.addTerms(1.0, self.x[0,j,r])
                self.model.addConstr(expr <= self.truckNum)

                expr.clear()   
            expr.clear()
            #9
            
            expr1 = LinExpr(0)
            expr2 = LinExpr(0)
           
            for r in R:
                for i in N0:

                    expr1.addTerms(1.0, self.x[i, self.customerNum + 1, r])
                for j in N_plus:
                    expr2.addTerms(1.0, self.x[0,j,r])
                self.model.addConstr(expr1 == expr2)
                expr1.clear()
                expr2.clear()
            
            
            #9
            for r in R:
                for i in C:
                    for j in N_plus:
                        if i!=j:
                            self.model.addConstr((self.u[i] - self.u[j] + 1) <= (self.customerNum  + 2)*( 1 - self.x[i,j,r]) )
            #10
            for i in N_plus:
                self.model.addConstr(1 <= self.u[i])
                self.model.addConstr(self.u[i] <= self.customerNum + 2)
                

In [13]:
path = "C:\\Users\\thebl\\Documents\\PDSTSP_MILP_new\\PDSTSP_10_customer_problems"
dirs = os.listdir(path)
problems_list = [file for file in dirs]

print(problems_list)

['20140813T111604.csv', '20140813T111606.csv', '20140813T111609.csv', '20140813T111611.csv', '20140813T111613.csv', '20140813T111615.csv', '20140813T111617.csv', '20140813T111619.csv', '20140813T111621.csv', '20140813T111623.csv', '20140813T111626.csv', '20140813T111628.csv', '20140813T111630.csv', '20140813T111632.csv', '20140813T111634.csv', '20140813T111636.csv', '20140813T111638.csv', '20140813T111640.csv', '20140813T111642.csv', '20140813T111644.csv', '20140813T111646.csv', '20140813T111649.csv', '20140813T111651.csv', '20140813T111653.csv', '20140813T111655.csv', '20140813T111657.csv', '20140813T111659.csv', '20140813T111701.csv', '20140813T111703.csv', '20140813T111705.csv', '20140813T111708.csv', '20140813T111710.csv', '20140813T111712.csv', '20140813T111714.csv', '20140813T111716.csv', '20140813T111718.csv', '20140813T111720.csv', '20140813T111722.csv', '20140813T111725.csv', '20140813T111727.csv', '20140813T111729.csv', '20140813T111731.csv', '20140813T111733.csv', '20140813T

In [14]:
data = Data()
data.model = Model("PDSTSP")
data.readData("C:\\Users\\thebl\\Documents\\PDSTSP_MILP_new\\PDSTSP_10_customer_problems\\" + problems_list[10])
data.addConstrs()

data.model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 219 rows, 229 columns and 1274 nonzeros
Model fingerprint: 0x1d705d38
Variable types: 1 continuous, 228 integer (218 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 110.3999999
Presolve removed 20 rows and 25 columns
Presolve time: 0.00s
Presolved: 199 rows, 204 columns, 1222 nonzeros
Found heuristic solution: objective 110.4000000
Variable types: 0 continuous, 204 integer (194 binary)

Root relaxation: objective 8.563560e+00, 55 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.56356    0   19  110.40000    8.56356  92.

In [15]:
data.model.printAttr('X')


    Variable            X 
-------------------------
  traveltime           31 
          u1            1 
          u2           10 
          u3            1 
          u4            1 
          u5           10 
          u6            9 
          u7            2 
          u8            2 
          u9            1 
         u10           11 
      x0,9,0            1 
     x5,10,0            1 
      x6,5,0            1 
      x8,6,0            1 
      x9,8,0            1 
      x0,1,1            1 
      x1,7,1            1 
     x2,10,1            1 
      x7,2,1            1 
        y0,3            1 
        y1,4            1 
